In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
from imageio import imread, imsave
from skimage.color import rgb2gray
from skimage.feature import corner_harris, corner_peaks, corner_fast, corner_subpix, ORB
import cv2
from sklearn.cluster import KMeans
from collections import Counter
from tabulate import tabulate

В лабораторной необходимо по положению ручек стула определить, можно ли задвинуть стул под/над стол или такой возможности нет. 
Ручки стула обернуты в зеленую ткань для контраста с материалом стола и стула.

Изображения разложены по папкам с названиями, соответствующими результату.

In [2]:
# Directories here - change name(!)
path_to_dirs = 'C:\\Users\\Daria\\Documents\\Обработка сигналов\\Big_lab\\'
dirs = ['above', 
        'below',
        'none']
name_dirs = ['Above', 'Below', 'None']

Для определения границ столешницы используется детектор Хафа от границ изображения: необходимо выделить верхнюю и нижнюю грани столешницы (горизонтальные линии).
После применения детектора имеем расстояния от левого верхнего угла изображения и углы между горизонтальной осью и этим расстоянием, по ним вычисляем высоту (от верхнего левого угла до точки, соответвующей началу линии по оси ОУ).

In [3]:
from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)
from skimage.feature import canny

def k_closest(sample, pivot, k):
    return sorted(sample, key=lambda i: abs(i - pivot))[:k]

def show_hough_transform(image):
     # Hough from Canny borders
    h, theta, d = hough_line(canny(image)) 
    
    # Find horizontal lines (dist + angle)
    h_up = image.shape[0]
    dist_hor = []
    for _, angle, dist in zip(*hough_line_peaks(h, theta, d)):
        if np.abs(angle) > 1:
            dist_hor.append((np.abs(dist), angle))       
     
    # Find heights from left corner
    heights = [np.abs(dist[0] / np.sin(dist[1])) for dist in dist_hor]
    
    # Find 2 closest among heights (here return closest for every height)
    result = [k_closest(heights, i, 2) for i in heights]
    # Find smallest distance
    distances = [(i, np.abs(i[0] - i[1])) for i in result]
    
    h_up, h_low = sorted(min(distances, key=lambda dist: dist[1])[0])

    return h_up, h_low

Определяем особые точки с помощью SIFT для определения точек на ручках (для этого и используется контрастный цвет).

In [5]:
# SIFT for special points searching
def sift_с(img):
    sift = cv2.SIFT_create()
    # Keypoints and descriptors.
    kp1, des1 = sift.detectAndCompute(img, None)
    
    return kp1

Здесь идет выделение 6 основных цветов картинки для последующего выделения ведущего зеленого цвета.

In [6]:
number_of_colors = 6
def find_colors(image):
    # Convert to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    modified_image = cv2.resize(image, (600, 400), interpolation = cv2.INTER_AREA)
    modified_image = modified_image.reshape(modified_image.shape[0]*modified_image.shape[1], 3)
    
    clf = KMeans(n_clusters = number_of_colors)
    labels = clf.fit_predict(modified_image)
    counts = Counter(labels)

    center_colors = clf.cluster_centers_
    # We get ordered colors by iterating through the keys
    ordered_colors = [center_colors[i] for i in counts.keys()]
    rgb_colors = [ordered_colors[i] for i in counts.keys()]
    return rgb_colors

Имеем 3 варианта для отсеивания зеленых точек:
* используем маску на основе отношения компонент: значения зеленого должны преобладать;
* для каждого выражения вычисляем 6 основных цветов, затем ищем среди них зеленый и рассматриваем цвета в его окрестности;
* вычисляем основные цвета только для первого фото, ищем зеленый в той же окрестности.

In [7]:
def is_green_threshold(img, pts, rgb):
    green_points_height = []
    for point in pts:
        if (int(point[0]) < img.shape[0] and int(point[1]) < img.shape[1]):
            (r, g, b) = img[int(point[0]), int(point[1])]
            if b <= g and r <= g:
                green_points_height.append(int(point[0]))
    return green_points_height
    
    
def is_green_clast(img, pts, b):
    green_points_height = []
    # Find main 6 colors of picture
    rgb = find_colors(img)
    # Green color found if max is in 1st coordinate(RGB)
    green_colors = []
    for i in rgb:
        i = i.tolist()
        if (i.index(max(i)) == 1):
            green_colors.append(np.round(i, 0))
    for point in pts:     
        if (int(point[0]) < img.shape[0] and int(point[1]) < img.shape[1]):
                (r, g, b) = img[int(point[0]), int(point[1])]
                for green in green_colors:
                    if np.abs(green[0] - r) <= 20 and np.abs(green[1] - g) <= 20 and np.abs(green[2] - b) <= 20:
                        green_points_height.append(int(point[0]))       
    return green_points_height


def is_green_from_first(img, pts, b):
    green_points_height = []
    green_colors = []
    # Green color found if max is in 1st coordinate(RGB)
    for i in rgb:
        i = i.tolist()
        if (i.index(max(i)) == 1):
            green_colors.append(np.round(i, 0))
        
    for point in pts:          
        if (int(point[0]) < img.shape[0] and int(point[1]) < img.shape[1]):
                (r, g, b) = img[int(point[0]), int(point[1])]
                for green in green_colors:
                    if np.abs(green[0] - r) <= 20 and np.abs(green[1] - g) <= 20 and np.abs(green[2] - b) <= 20:
                        green_points_height.append(int(point[0]))       
    return green_points_height


define_green = [is_green_clast, is_green_threshold, is_green_from_first]

Среди особых точек ищем зеленые тремя разными методами.
После прохода по особым точкам и отделения зеленых находим наименьшую и наибольшую высоту от края изображения.

In [8]:
def get_green_point_height(kp, img, rgb, is_green):
    # Convert key points to coordinates
    pts = cv2.KeyPoint_convert(kp)
    # Round coordinates to get pixel
    pts = np.round(pts, 0)
            
    
    # Here find height of green key points
    green_points_height = is_green(img, pts, rgb)

    if len(green_points_height) == 0:
        return -1, -1
    max_h = np.min(green_points_height)
    min_h = np.max(green_points_height)

    return max_h, min_h

Интересуют случай, когда верхняя точка зеленой ручки по высоте больше
нижнего края столешницы («пройдет ниже») и случай, когда нижняя точка меньше верхней 
грани столешницы («пройдет выше»), иначе получили «не пройдет».
Стоит отметить, что высота считается от верхнего края!

Выведена таблица для трех различных методов и точностями определения 3-х возможных случаев и точности в целом.

In [9]:
rows = []
headers = ['Method', 'Accuracy above', 'Accuracy below', 'Impossible accuracy', 'Total accuracy']

for method in define_green:
    acc = []
    acc.clear()
    i = 0
    for dir_ in dirs:
        dir_actual = os.path.join(path_to_dirs, dir_)       
        os.chdir(dir_actual)
        images = [img_ for img_ in os.listdir(dir_actual)
                  if img_.endswith(".jpg") or
                  img_.endswith(".jpeg") or
                  img_.endswith("png")]
        sum_acc = 0
        for j in range(len(images)):
            image = imread(images[j])
            if j == 0:
                rgb = find_colors(image)
            gray = rgb2gray(image)
            table_up, table_low = show_hough_transform(gray)
            key_points = sift_с(image)
            chair_up, chair_low = get_green_point_height(key_points, image, rgb, method)
            if chair_low < 0 or chair_up < 0: 
                res = 'none'      
            elif chair_low < table_up: 
                res = 'above'
            elif chair_up > table_low:
                res = 'below'
            else:
                res = 'none'
            if res == name_dirs[i].lower():
                sum_acc += 1
        acc.append(sum_acc/len(images))
        i += 1
    rows.append((method.__name__, acc[0], acc[1], acc[2], np.round(np.sum(acc) / 3, decimals=3)))
print(tabulate(rows, headers, tablefmt="pipe"))

| Method              |   Accuracy above |   Accuracy below |   Impossible accuracy |   Total accuracy |
|:--------------------|-----------------:|-----------------:|----------------------:|-----------------:|
| is_green_clast      |              0.3 |              0.3 |                   1   |            0.533 |
| is_green_threshold  |              0.1 |              0.2 |                   0.8 |            0.367 |
| is_green_from_first |              0.4 |              0.5 |                   0.4 |            0.433 |


Во время решения задачи стало понятно, что попытка положиться на определение цвета была не совсем удачной. Под разным углом(поскольку стол перед окном, свет так или иначе падат по-разному) зеленый цвет, на который была опора во время решения, может иметь больший отлив в синие оттенки, что не позволяет использовать цветовые палитры корректно - ручной подбор границ цвета для всех изображений не дал желаемых результатов.

Видно, что лучшие результаты показывает определение основных цветов изображения на каждом из них. Это оптимальный вариант для решения, поскольку может учитывать тонкости оттенков.
Попытки сортировать точки по соотношению цветов между ними были не самыми удачными в силу выше описанного, равно как и кластеризация по цветам в начале.